In [1]:
from sklearn.datasets import make_classification
import torch


In [2]:
X, y = make_classification(n_samples=10, n_features=2,n_informative=2, n_redundant=0, n_classes=2, random_state=42)

In [3]:
X


array([[ 1.06833894, -0.97007347],
       [-1.14021544, -0.83879234],
       [-2.8953973 ,  1.97686236],
       [-0.72063436, -0.96059253],
       [-1.96287438, -0.99225135],
       [-0.9382051 , -0.54304815],
       [ 1.72725924, -1.18582677],
       [ 1.77736657,  1.51157598],
       [ 1.89969252,  0.83444483],
       [-0.58723065, -1.97171753]])

In [4]:
X.shape

(10, 2)

In [5]:
y.shape

(10,)

In [6]:
X = torch.from_numpy(X)
y = torch.from_numpy(y)

In [7]:
X

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388],
        [-2.8954,  1.9769],
        [-0.7206, -0.9606],
        [-1.9629, -0.9923],
        [-0.9382, -0.5430],
        [ 1.7273, -1.1858],
        [ 1.7774,  1.5116],
        [ 1.8997,  0.8344],
        [-0.5872, -1.9717]], dtype=torch.float64)

In [8]:
from torch.utils.data import DataLoader, Dataset

In [17]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        #transformation can be applied here
        return self.X[idx], self.y[idx]

In [18]:
dataset = CustomDataset(X, y)

In [19]:
len(dataset)

10

In [20]:
dataset[0]

(tensor([ 1.0683, -0.9701], dtype=torch.float64), tensor(1))

In [23]:
dataloader = DataLoader(dataset ,  batch_size= 2, shuffle= True)# num_workers for parallel data loading

In [24]:
for batch_features, batch_labels in dataloader:
    print(batch_features)
    print(batch_labels)
    print("-"*50)    

tensor([[-1.9629, -0.9923],
        [-0.5872, -1.9717]], dtype=torch.float64)
tensor([0, 0])
--------------------------------------------------
tensor([[ 1.7273, -1.1858],
        [ 1.7774,  1.5116]], dtype=torch.float64)
tensor([1, 1])
--------------------------------------------------
tensor([[-2.8954,  1.9769],
        [-0.9382, -0.5430]], dtype=torch.float64)
tensor([0, 1])
--------------------------------------------------
tensor([[ 1.0683, -0.9701],
        [-0.7206, -0.9606]], dtype=torch.float64)
tensor([1, 0])
--------------------------------------------------
tensor([[-1.1402, -0.8388],
        [ 1.8997,  0.8344]], dtype=torch.float64)
tensor([0, 1])
--------------------------------------------------


In [30]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import pandas as pd

In [31]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [32]:
df.drop(['Unnamed: 32', 'id'], axis=1, inplace=True)

In [33]:
X = df.drop('diagnosis', axis=1).values
y = df['diagnosis'].values

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [36]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)


In [39]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)


In [40]:
from torch.utils.data import DataLoader, Dataset
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [41]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [43]:
train_dataset[10]

(array([-0.4976419 ,  0.61365274, -0.49813131, -0.53102815, -0.57694824,
        -0.17494424, -0.36215622, -0.284859  ,  0.43345165,  0.17818232,
        -0.36844966,  0.55310406, -0.31671104, -0.40524636,  0.04025752,
        -0.03795529, -0.18043065,  0.16478901, -0.12170969,  0.23079329,
        -0.50044002,  0.81940367, -0.46922838, -0.53308833, -0.04910117,
        -0.04160193, -0.14913653,  0.09681787,  0.10617647,  0.49035329]),
 np.int64(0))

In [44]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [45]:
import torch.nn as nn

class MySimpleNN(nn.Module):
    def __init__(self, input_size):
        super(MySimpleNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        out = self.network(x)
        return out

In [46]:
learning_rate = 0.1
epochs =100

In [47]:
model = MySimpleNN(input_size=X_train.shape[1])
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [48]:
for epoch in range(epochs):
    for batch_features, batch_labels in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_features.float())
        loss = criterion(outputs.squeeze(), batch_labels.float())
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.1351
Epoch [20/100], Loss: 0.0389
Epoch [30/100], Loss: 0.0052
Epoch [40/100], Loss: 0.0020
Epoch [50/100], Loss: 0.0099
Epoch [60/100], Loss: 0.0006
Epoch [70/100], Loss: 0.0016
Epoch [80/100], Loss: 0.0052
Epoch [90/100], Loss: 0.0007
Epoch [100/100], Loss: 0.0002


In [49]:
model.eval()
accuracy_list = []
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        y_pred = model(batch_features.float())
        y_pred = (y_pred > 0.8).float()
        
        batch_accuracy = (y_pred.squeeze() == batch_labels.float()).float().mean().item()
        accuracy_list.append(batch_accuracy)

overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Overall Accuracy: {overall_accuracy:.4f}')


Overall Accuracy: 0.9766
